In [65]:
import pandas as pd 
import numpy as np 
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

In [26]:
df = pd.read_csv("HousePrices.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   rownames    546 non-null    int64 
 1   price       546 non-null    int64 
 2   lotsize     546 non-null    int64 
 3   bedrooms    546 non-null    int64 
 4   bathrooms   546 non-null    int64 
 5   stories     546 non-null    int64 
 6   driveway    546 non-null    object
 7   recreation  546 non-null    object
 8   fullbase    546 non-null    object
 9   gasheat     546 non-null    object
 10  aircon      546 non-null    object
 11  garage      546 non-null    int64 
 12  prefer      546 non-null    object
dtypes: int64(7), object(6)
memory usage: 55.6+ KB


In [27]:
df.head()

,rownames,price,lotsize,bedrooms,bathrooms,stories,driveway,recreation,fullbase,gasheat,aircon,garage,prefer
0,1,42000,5850,3,1,2,yes,no,yes,no,no,1,no
1,2,38500,4000,2,1,1,yes,no,no,no,no,0,no
2,3,49500,3060,3,1,1,yes,no,no,no,no,0,no
3,4,60500,6650,3,1,2,yes,yes,no,no,no,0,no
4,5,61000,6360,2,1,1,yes,no,no,no,no,0,no


In [28]:
cat_cols = ['driveway', 'recreation', 'fullbase', 'gasheat', 'aircon','prefer']
for col in cat_cols:
    df[col] = df[col].map({'yes': 1, 'no': 0})

In [35]:
df[[col for col in df.columns if col != "rownames"]].describe().round(2)


,price,lotsize,bedrooms,bathrooms,stories,driveway,recreation,fullbase,gasheat,aircon,garage,prefer
count,546.00,546.00,546.00,546.00,546.00,546.00,546.00,546.00,546.00,546.00,546.00,546.00
mean,68121.60,5150.27,2.97,1.29,1.81,0.86,0.18,0.35,0.05,0.32,0.69,0.23
std,26702.67,2168.16,0.74,0.50,0.87,0.35,0.38,0.48,0.21,0.47,0.86,0.42
min,25000.00,1650.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,49125.00,3600.00,2.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,62000.00,4600.00,3.00,1.00,2.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
75%,82000.00,6360.00,3.00,2.00,2.00,1.00,0.00,1.00,0.00,1.00,1.00,0.00
max,190000.00,16200.00,6.00,4.00,4.00,1.00,1.00,1.00,1.00,1.00,3.00,1.00


In [39]:
df.groupby('prefer').describe()[["price","lotsize"]]

price                                                                  \
        count          mean           std      min      25%      50%      75%   
prefer                                                                          
0       418.0  63263.485646  24214.231520  25000.0  46625.0  58500.0  73000.0   
1       128.0  83986.367188  28342.926632  31900.0  64975.0  79750.0  95000.0   

                 lotsize                                                    \
             max   count         mean          std     min     25%     50%   
prefer                                                                       
0       175000.0   418.0  4868.832536  2021.201021  1650.0  3500.0  4310.0   
1       190000.0   128.0  6069.320312  2376.429950  1950.0  4380.0  6360.0   

                          
            75%      max  
prefer                    
0       6000.00  16200.0  
1       7103.75  13200.0

In [33]:
for col in cat_cols:
    display(pd.DataFrame((df[col].value_counts(normalize=True)).reset_index()*100).round(2))

,driveway,proportion
0,100,85.9
1,0,14.1


,recreation,proportion
0,0,82.23
1,100,17.77


,fullbase,proportion
0,0,65.02
1,100,34.98


,gasheat,proportion
0,0,95.42
1,100,4.58


,aircon,proportion
0,0,68.32
1,100,31.68


,prefer,proportion
0,0,76.56
1,100,23.44


In [67]:
def weights(df):

    X = df.drop(columns=['prefer','rownames'])
    y = df['prefer']

    model = LogisticRegression()
    model.fit(X, y)

    coefficients = model.coef_[0]
    intercept = model.intercept_[0]

    result =  pd.DataFrame({'variable': X.columns, 'weight': coefficients})
    result["weight"] = result["weight"].abs()
    
    return result
   

In [68]:
display(weights(df).sort_values(by="weight",ascending=False))

,variable,weight
2,bedrooms,0.686244
3,bathrooms,0.667410
7,fullbase,0.566442
10,garage,0.387807
6,recreation,0.294395
8,gasheat,0.215843
4,stories,0.144576
5,driveway,0.050804
9,aircon,0.026022
0,price,0.000037


In [73]:
#Varianza
pd.DataFrame(np.var(df.select_dtypes(include=np.number)))


,0
rownames,2.484292e+04
price,7.117267e+08
lotsize,4.692303e+06
bedrooms,5.427451e-01
bathrooms,2.517007e-01
stories,7.523950e-01
driveway,1.211374e-01
recreation,1.460941e-01
fullbase,2.274450e-01
gasheat,4.369105e-02


In [84]:
df.select_dtypes(include='number').cov().to_csv("ola.csv")